# Análise de Conteúdo - Qatar

Análise do Conteúdo dos Tweets dos veículos jornalísticos sobre a cobertura jornalística no Qatar.

Trabalho desenvolvido para a disciplina Mineiração de Dados Não Estruturados, do curso de Pós-graduação em Jornalismo de Dados, Automação e Data Storytelling.

Código executado no Google Colab

**Grupo**: Ananda Ridart, Gabriel Ronan, Géssica Brandino e Vitor Arthur

**Professora**: Clarisse Castellã

# Etapa 1 - Importações e Requisições no Twitter

## Primeiros passos

In [ ]:
# instalando twint no colab
!git clone --depth=1 https://github.com/twintproject/twint.git
!cd /content/twint && pip3 install . -r requirements.txt && pip install aiohttp==3.7.0 --force-reinstal

In [ ]:
# instalando outras bibliotecas
!pip install nest-asyncio
!pip install nltk

In [ ]:
# importando bibliotecas
from google.colab import drive
drive.mount('/content/drive')
import os
import pandas as pd
#import twint
#import nest_asyncio
from os import path 
from re import A
from sklearn.model_selection import train_test_split

# baixando recursos para processamento de linguagem natural
import nltk
from nltk.tokenize import word_tokenize
from nltk.classify import NaiveBayesClassifier, MaxentClassifier, SklearnClassifier
from sklearn.svm import LinearSVC
nltk.download('punkt')
nltk.download('stopwords')


## Coleta



### Fazendo scraper para recolher tweets do Globo Esporte
Utilizando a palavra-chave: "Qatar", "Catar" e "Copa"

In [ ]:
# pre-requisito para o twint funcionar
nest_asyncio.apply()

# criando requisição usando twint
c = twint.Config()
c.Limit = 500  # quantidade de tweets
c.Store_csv = True  # arquivo em csv
c.Search = 'Catar'  # query - "Qatar", "Catar" e "Copa"
c.Username = 'geglobo'  # perfil que será observado
text_file = '/content/drive/MyDrive/Insper/projeto_mineiracao/catar_globo.csv' 
c.Output = text_file

# rodando requsição
twint.run.Search(c)

### Fazendo scraper para recolher tweets do LanceNet
Utilizando a palavra-chave: "Qatar", "Catar" e "Copa"

In [ ]:
# pre-requisito para o twint funcionar
nest_asyncio.apply()

# criando requisição usando twint
d = twint.Config()
d.Limit = 500  # quantidade de tweets
d.Store_csv = True  # arquivo em csv
d.Search = 'Qatar'  # query - "Qatar", "Catar" e "Copa"
d.Username = 'lancenet'  # perfil que será observado
text_file = '/content/drive/MyDrive/Insper/projeto_mineiracao/catar_lancenet.csv' 
d.Output = text_file

# rodando requsição
twint.run.Search(d)

### Fazendo scraper para recolher tweets do Uol Esportes
Utilizando a palavra-chave: "Qatar", "Catar" e "Copa"

In [ ]:
# pre-requisito para o twint funcionar
nest_asyncio.apply()

# criando requisição usando twint
e = twint.Config()
e.Limit = 500  # quantidade de tweets
e.Store_csv = True  # arquivo em csv
e.Search = 'Qatar'  # query - "Qatar", "Catar" e "Copa"
e.Username = 'UOLEsporte'  # perfil que será observado
text_file = '/content/drive/MyDrive/Insper/projeto_mineiracao/catar_uol.csv' 
e.Output = text_file

# rodando requsição
twint.run.Search(d)

### Fazendo scraper para recolher tweets do ESPM
Utilizando a palavra-chave: "Qatar", "Catar" e "Copa"

In [ ]:
# pre-requisito para o twint funcionar
nest_asyncio.apply()

# criando requisição usando twint
f = twint.Config()
f.Limit = 500  # quantidade de tweets
f.Store_csv = True  # arquivo em csv
f.Search = 'Copa'  # query - "Qatar", "Catar" e "Copa"
f.Username = 'ESPNBrasil'  # perfil que será observado
text_file = '/content/drive/MyDrive/Insper/projeto_mineiracao/copa_espn.csv' 
f.Output = text_file

# rodando requsição
twint.run.Search(f)

# Etapa 2: Criando Dataframe principal

## Juntando todos os dataframes de coleta

In [ ]:
# lendo dataframes
def leitura_dataframe(caminho = list(), palavra_chave = list()):
  # lista de dataframes
  dfs = []
  contc = 0

  # lendo um dataframe por vez
  for c in caminho:
    for p in palavra_chave:
      df = pd.read_csv(c[contc], sep = ',')
      df['palavra_chave'] = p
      dfs.append(df)
      contc += 1
      if contc == 3:
        contc = 0
  
  df = pd.concat(dfs)
  df = df.reset_index()
  return df

# MAIN
# listas para iterar os caminhos, nomes e palavras-chave
caminhos = [['/content/drive/MyDrive/Insper/projeto_mineiracao/catar_globo.csv',
             '/content/drive/MyDrive/Insper/projeto_mineiracao/qatar_globo.csv',
             '/content/drive/MyDrive/Insper/projeto_mineiracao/copa_globo.csv'],
            ['/content/drive/MyDrive/Insper/projeto_mineiracao/catar_lance.csv', 
             '/content/drive/MyDrive/Insper/projeto_mineiracao/qatar_lance.csv',
             '/content/drive/MyDrive/Insper/projeto_mineiracao/copa_lance.csv'],
            ['/content/drive/MyDrive/Insper/projeto_mineiracao/catar_uol.csv',
             '/content/drive/MyDrive/Insper/projeto_mineiracao/qatar_uol.csv', 
             '/content/drive/MyDrive/Insper/projeto_mineiracao/copa_uol.csv'],
            ['/content/drive/MyDrive/Insper/projeto_mineiracao/catar_espn.csv',
             '/content/drive/MyDrive/Insper/projeto_mineiracao/qatar_espn.csv',
             '/content/drive/MyDrive/Insper/projeto_mineiracao/copa_espn.csv']]
palavras = ['catar', 'qatar', 'copa']

# iterável com caminhos para leitura dos datasets
df_total = leitura_dataframe(caminhos, palavras)

## Retirando repetições para finalizar o dataset

In [ ]:
# mostrando dataframe com todas as coletas
df_total.head(10)

In [ ]:
df_total.to_csv('tweets_catar.csv')

In [ ]:
# mostrando as colunas do dataset
df_total.columns

In [ ]:
# retirando repetições dos tweets
df_total.drop_duplicates(subset=['tweet'], inplace=True)

In [ ]:
df_total

,index,id,conversation_id,created_at,date,time,timezone,user_id,username,name,...,source,user_rt_id,user_rt,retweet_id,reply_to,retweet_date,translate,trans_src,trans_dest,palavra_chave
0,0,1598742544690679821,1598742544690679821,2022-12-02 18:15:00 UTC,2022-12-02,18:15:00,0,10842792,geglobo,ge,...,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN,catar
1,1,1598738771108765696,1598738771108765696,2022-12-02 18:00:00 UTC,2022-12-02,18:00:00,0,10842792,geglobo,ge,...,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN,catar
2,2,1598692584158859265,1598692584158859265,2022-12-02 14:56:28 UTC,2022-12-02,14:56:28,0,10842792,geglobo,ge,...,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN,catar
3,3,1598692319640829959,1598692319640829959,2022-12-02 14:55:25 UTC,2022-12-02,14:55:25,0,10842792,geglobo,ge,...,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN,catar
4,4,1598660543031566337,1598660543031566337,2022-12-02 12:49:09 UTC,2022-12-02,12:49:09,0,10842792,geglobo,ge,...,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN,catar
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2583,494,1596300665412030464,1596300665412030464,2022-11-26 00:31:51 UTC,2022-11-26,00:31:51,0,51124892,espnbrasil,ESPN Brasil,...,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN,copa
2584,495,1596279860900397056,1596279860900397056,2022-11-25 23:09:10 UTC,2022-11-25,23:09:10,0,51124892,espnbrasil,ESPN Brasil,...,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN,copa
2585,496,1596279387858550784,1596279387858550784,2022-11-25 23:07:18 UTC,2022-11-25,23:07:18,0,51124892,espnbrasil,ESPN Brasil,...,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN,copa
2586,497,1596277690444967936,1596277690444967936,2022-11-25 23:00:33 UTC,2022-11-25,23:00:33,0,51124892,espnbrasil,ESPN Brasil,...,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN,copa


In [ ]:
# retirando repetições dos tweets e preparando para fazer amostragem para classificação manual
df_total.drop_duplicates(subset=['tweet'], inplace=True)
tweet_para_amostra = df_total[['tweet', 'username', 'palavra_chave']]
tweet_para_amostra

,tweet,username,palavra_chave
0,#CopadasCamisas Qual a camisa mais bonita da...,geglobo,catar
1,#CopadasCamisas Qual a camisa mais bonita da...,geglobo,catar
2,#CopadasCamisas Qual a camisa mais bonita da...,geglobo,catar
3,#CopadasCamisas Qual a camisa mais bonita da...,geglobo,catar
4,👋🚌 Ônibus da Alemanha 🇩🇪 a caminho do aeropor...,geglobo,catar
...,...,...,...
2583,Mexicanos respondem: Dani Alves mereceu ser co...,espnbrasil,copa
2584,Jogador de Portugal na Copa do Mundo acompanha...,espnbrasil,copa
2585,Inglaterra termina jogo vaiada pela própria to...,espnbrasil,copa
2586,Neto corneta possível titularidade de Dani Alv...,espnbrasil,copa


## Criando uma amostra para o corpus de treino
Realizando uma Amostragem Aleatória Simples

In [ ]:
# realizando amostra
amostra = tweet_para_amostra.sample(180)
amostra['classificação'] = ''
amostra.to_csv("corpus_treino.csv")

# Etapa 3 - Treinando a máquina

## Tratamento do corpus de treino
Tokenizando o corpus e retirando stopwords

In [ ]:
# recebendo arquivos de treino
neutro = '/content/drive/MyDrive/Insper/projeto_mineiracao/treino/treino_neutro.data'
positivo = '/content/drive/MyDrive/Insper/projeto_mineiracao/treino/treino_positivo.data'
negativo = '/content/drive/MyDrive/Insper/projeto_mineiracao/treino/treino_negativo.data'

In [ ]:
# Função para fazer tokenização das frases
def tokenizar(entrada):
  lista_token = list()
  for palavra in entrada:
    token = word_tokenize(palavra)
    lista_token.append(token)
  return lista_token

In [ ]:
# retirando os stopwords
def tirar_stop(entrada):

  # criando variáveis
  lista_palavras = list()
  final = list()

  # listando as stopwords
  stopwords = nltk.corpus.stopwords.words('portuguese')
  outras_stop = ['!', ':', '#', 'https', ',', '?', '.', '-', '(', ')', '@', 't.co']
  for stop in outras_stop:
    stopwords.append(stop)

  # retirando as stopwords
  for palavra in entrada:
    if palavra not in stopwords:
      palavra.replace('.co','.')
      palavra.replace('//t.co','.')
      lista_palavras.append(palavra)
  
  # retirando parte dos links
  for i in lista_palavras:
    p = i.replace('.co','.')
    p = i.replace('//t.co','.')
    final.append(p)
  
  return final

In [ ]:
# Criação de Bag of words
def bag(entrada):
  bag_of_words = dict()

  # criando features
  for p in entrada:
    bag_of_words[p] = entrada.count(p)
  return bag_of_words

## Treinando Algoritimo de Aprendizado de Máquina Supervisionado

In [ ]:
# treinando a máquina
def treinamento(positivo, negativo, neutro):

  # criando listas
  pos_dados = list()
  neg_dados = list()
  neu_dados = list()

  # alocando os arquivos de treino
  # positivo
  with open(positivo, 'r') as arquivo:
    linhas = arquivo.readlines()
    for linha in linhas:
      if len(linha) > 0:
        pos_dados.append(linha.lower())
  
  # negativo
  with open(negativo, 'r') as arquivo:
    linhas = arquivo.readlines()
    for linha in linhas:
      if len(linha) > 0:
        neg_dados.append(linha.lower()) 

  # neutro
  with open(neutro, 'r') as arquivo:
    linhas = arquivo.readlines()
    for linha in linhas:
      if len(linha) > 0:
        neu_dados.append(linha.lower())

  # tokenizando palavras, retirando stopwords e retirando as bags of words
  posfeats = list()
  for l in tokenizar(pos_dados):
    sem_stop = tirar_stop(l)
    posfeats.append((bag(sem_stop), 'pos'))
  print(posfeats)

  negfeats = list()
  for l in tokenizar(neg_dados):
    sem_stop = tirar_stop(l)
    negfeats.append((bag(sem_stop), 'neg'))
  print(negfeats)

  neufeats = list()
  for l in tokenizar(neu_dados):
    sem_stop = tirar_stop(l)
    neufeats.append((bag(sem_stop), 'neu'))
  print(neufeats)

  # criando string de treino
  treino = posfeats + negfeats + neufeats

  # Rodando os algoritmos para treina-los
  classificadorME = MaxentClassifier.train(treino, 'GIS', trace=0, 
                                           encoding=None, labels=None, 
                                           gaussian_prior_sigma=0, max_iter = 1)
  
  #SVM
  classificadorSVM = SklearnClassifier(LinearSVC(), sparse=False)
  classificadorSVM.train(treino)

  # Naive Bayes
  classificadorNB = NaiveBayesClassifier.train(treino)

  return ([classificadorME, classificadorSVM, classificadorNB])

In [ ]:
# Rodar algoritmo 
classificadores = treinamento(positivo, negativo, neutro)

[({'walter': 1, 'casagrande': 1, 'jr.': 1, 'após': 1, 'dois': 1, 'jogos': 1, 'espanha': 1, 'frança': 1, 'seleções': 1, 'favoritas': 1, 'copa': 1, './h2kif3oh6x': 1}, 'pos'), ({'emocionar': 1, 'partida': 1, 'mata-mata': 1, 'fase': 1, 'grupos': 1, 'senegal': 1, 'derrota': 1, 'equador': 1, 'segue': 1, 'vivo': 1, 'copa': 1, 'mundo': 1, 'lancecopa': 1, 'copadomundofifa': 1, 'qatar2022': 1, '📷manan': 1, 'vatsyayana': 1, '/': 4, 'afp': 4, '📷raul': 1, 'arboleda': 1, '📷ozan': 1, 'kose': 1, '📷jung': 1, 'yeon-je': 1, './cgim9k6kjg': 1}, 'pos'), ({'meia': 1, 'costa': 1, 'rica': 1, 'mantém': 1, 'esperança': 1, 'classificação': 1, 'oitavas': 1, 'copa': 1, './uf4zatzubp': 1}, 'pos'), ({'modelo': 1, 'croata': 1, 'faz': 1, 'sucesso': 1, 'visual': 1, 'copa': 1, 'qatar': 1, ';': 1, 'escolha': 1, 'melhor': 1, 'fifaworldcup': 1, 'qatar2022': 1, './vttz4vxkxm': 1}, 'pos'), ({'números': 1, '🏴\U000e0067\U000e0062\U000e0065\U000e006e\U000e0067\U000e007f⚽': 1, 'harry': 1, 'kane': 1, 'brilha': 1, 'copa': 1, 'mun

# Etapa 4 - Implementação do Algoritmo de Aprendizado

## Preparando um Dataframe para fazer a análise

In [ ]:
# Dataframe que será utilizado para armazenar a classificação
df_classificacao = df_total[['user_id', 'username', 'tweet', 'link', 'palavra_chave']]

In [ ]:
# Tokenizando as palavras
df_classificacao['bag_of_words'] = tokenizar(df_classificacao['tweet'])

# Retirando Stopwords
limpo = list()
for l in df_classificacao['bag_of_words']:
  ss = tirar_stop(l)
  limpo.append(ss)
df_classificacao['bag_of_words'] = limpo

# Criando bag of words
dfbag_of_words = list()
for l in df_classificacao['bag_of_words']:
  bow = bag(l)
  dfbag_of_words.append(bow)
df_classificacao['bag_of_words'] = dfbag_of_words

## Aplicando o modelo que foi treinado

In [ ]:
# criando função para classificar tweets
def classificar(entrada, nome, coluna_dataframe, classificador):
  classificacao = list()
  for l in entrada[coluna_dataframe]:
    classificado = classificadores[classificador].classify(l)
    classificacao.append(classificado)
  entrada[nome] = classificacao


## MAIN
classificar(df_classificacao, 'classificacao_MaxentClassifier', 'bag_of_words', 0)
classificar(df_classificacao, 'classificacao_SklearnClassifier', 'bag_of_words', 1)
classificar(df_classificacao, 'classificacao_NaiveBayesClassifier', 'bag_of_words', 2)

In [ ]:
# mostrando Dataframe final com as classificações automáticas aplicadas
df_classificacao.to_csv('mineração_arquivo_classificado.csv')